In [1]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
from numpy import load

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
np.random.seed(40)
tf.random.set_seed(40)

In [4]:
def makeColumns(df,columns):
  df.index = df.index.rename('Timestamp')
  df.columns = columns
  return df
def changeDtypes(df,columns):
  print(columns)
  if 'CAN ID' in df:
    df['CAN ID'] = df['CAN ID'].apply(lambda x: int(str(x),16))
  if 'DLC' in df:
    df['DLC'] = df['DLC'].astype('int32')
  if 'RTR' in df:
    df['RTR'] = df['RTR'].astype('int32')
  for col in columns:
    df[col] = df[col].apply(lambda x: int(str(x),16) )

  return df

normal_run_data 전처리

<데이터 분류>
['Timestamp','CAN ID','RTR','DLC','Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8'] 

-NaN 처리

-데이터 타입 int형으로 통일

In [ ]:
#df.columns = ['CAN ID','RTR','DLC','Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8'] 

normaldf = pd.read_csv('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/normal_run_data.txt',delimiter = '\t') 
normaldf.columns

Index(['Timestamp: 1479121434.850202        ID: 0350    000    DLC: 8    05 28 84 66 6d 00 00 a2'], dtype='object')

In [ ]:
with open('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/normal_run_data.txt','r') as file:
  for line in file.readlines(1080):
    print(line)

Timestamp: 1479121434.850202        ID: 0350    000    DLC: 8    05 28 84 66 6d 00 00 a2

Timestamp: 1479121434.850423        ID: 02c0    000    DLC: 8    14 00 00 00 00 00 00 00

Timestamp: 1479121434.850977        ID: 0430    000    DLC: 8    00 00 00 00 00 00 00 00

Timestamp: 1479121434.851215        ID: 04b1    000    DLC: 8    00 00 00 00 00 00 00 00

Timestamp: 1479121434.851463        ID: 01f1    000    DLC: 8    00 00 00 00 00 00 00 00

Timestamp: 1479121434.851711        ID: 0153    000    DLC: 8    00 00 00 ff 00 ff 00 00

Timestamp: 1479121434.851963        ID: 0002    000    DLC: 8    00 00 00 00 00 00 00 0a

Timestamp: 1479121434.852202        ID: 018f    000    DLC: 8    fe 36 00 00 00 3c 00 00

Timestamp: 1479121434.852443        ID: 0130    000    DLC: 8    03 80 00 ff 21 80 00 9d

Timestamp: 1479121434.852687        ID: 0131    000    DLC: 8    00 80 00 00 2d 7f 00 97

Timestamp: 1479121434.852938        ID: 0140    000    DLC: 8    00 00 00 00 14 20 20 e6

Timestamp:

In [ ]:
normaldict = {}

with open('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/normal_run_data.txt','r') as file:
  for line in file.readlines():
    line = line.split()
    if 'ID:' not in line:
      continue
    line.remove('Timestamp:')
    line.remove('ID:')
    line.remove('DLC:')
    normaldict[float(line[0])] = line[1:]



In [ ]:
#creating dataframe from dictionary dfdict
normaldf = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in normaldict.items()]))
normaldf = normaldf.T
normaldf.head(30)

,0,1,2,3,4,5,6,7,8,9,10
1.479121e+09,0350,000,8,05,28,84,66,6d,00,00,a2
1.479121e+09,02c0,000,8,14,00,00,00,00,00,00,00
1.479121e+09,0430,000,8,00,00,00,00,00,00,00,00
1.479121e+09,04b1,000,8,00,00,00,00,00,00,00,00
1.479121e+09,01f1,000,8,00,00,00,00,00,00,00,00
1.479121e+09,0153,000,8,00,00,00,ff,00,ff,00,00
1.479121e+09,0002,000,8,00,00,00,00,00,00,00,0a
1.479121e+09,018f,000,8,fe,36,00,00,00,3c,00,00
1.479121e+09,0130,000,8,03,80,00,ff,21,80,00,9d
1.479121e+09,0131,000,8,00,80,00,00,2d,7f,00,97


In [ ]:
normaldf.index = normaldf.index.rename('Timestamp')
normaldf.columns =['CAN ID','RTR','DLC','Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8'] 
normaldf.head()

,CAN ID,RTR,DLC,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Data8
Timestamp,,,,,,,,,,,
1.479121e+09,0350,000,8,05,28,84,66,6d,00,00,a2
1.479121e+09,02c0,000,8,14,00,00,00,00,00,00,00
1.479121e+09,0430,000,8,00,00,00,00,00,00,00,00
1.479121e+09,04b1,000,8,00,00,00,00,00,00,00,00
1.479121e+09,01f1,000,8,00,00,00,00,00,00,00,00


In [ ]:
normaldf.dtypes

CAN ID    object
RTR       object
DLC       object
Data1     object
Data2     object
Data3     object
Data4     object
Data5     object
Data6     object
Data7     object
Data8     object
dtype: object

In [ ]:
normaldf = normaldf.fillna(0)
normaldf.head(30)

,CAN ID,RTR,DLC,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Data8
Timestamp,,,,,,,,,,,
1.479121e+09,0350,000,8,05,28,84,66,6d,00,00,a2
1.479121e+09,02c0,000,8,14,00,00,00,00,00,00,00
1.479121e+09,0430,000,8,00,00,00,00,00,00,00,00
1.479121e+09,04b1,000,8,00,00,00,00,00,00,00,00
1.479121e+09,01f1,000,8,00,00,00,00,00,00,00,00
1.479121e+09,0153,000,8,00,00,00,ff,00,ff,00,00
1.479121e+09,0002,000,8,00,00,00,00,00,00,00,0a
1.479121e+09,018f,000,8,fe,36,00,00,00,3c,00,00
1.479121e+09,0130,000,8,03,80,00,ff,21,80,00,9d


In [ ]:

normaldf['CAN ID'] = normaldf['CAN ID'].apply(lambda x: int(str(x),16))
normaldf['DLC'] = normaldf['DLC'].astype('int32')
normaldf['RTR'] = normaldf['RTR'].astype('int32')
for col in ['Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8'] :
  normaldf[col] = normaldf[col].apply(lambda x: int(str(x),16) )

normaldf.head(10)

,CAN ID,RTR,DLC,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Data8
Timestamp,,,,,,,,,,,
1.479121e+09,848,0,8,5,40,132,102,109,0,0,162
1.479121e+09,704,0,8,20,0,0,0,0,0,0,0
1.479121e+09,1072,0,8,0,0,0,0,0,0,0,0
1.479121e+09,1201,0,8,0,0,0,0,0,0,0,0
1.479121e+09,497,0,8,0,0,0,0,0,0,0,0
1.479121e+09,339,0,8,0,0,0,255,0,255,0,0
1.479121e+09,2,0,8,0,0,0,0,0,0,0,10
1.479121e+09,399,0,8,254,54,0,0,0,60,0,0
1.479121e+09,304,0,8,3,128,0,255,33,128,0,157


In [ ]:
normaldf.dtypes

CAN ID    int64
RTR       int32
DLC       int32
Data1     int64
Data2     int64
Data3     int64
Data4     int64
Data5     int64
Data6     int64
Data7     int64
Data8     int64
dtype: object

In [ ]:
normaldf.insert(11,"Label",0)
normaldf = normaldf.drop("RTR",axis=1)

In [ ]:
normaldf.head(100)

,CAN ID,DLC,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Data8,Label
Timestamp,,,,,,,,,,,
1.479121e+09,848,8,5,40,132,102,109,0,0,162,0
1.479121e+09,704,8,20,0,0,0,0,0,0,0,0
1.479121e+09,1072,8,0,0,0,0,0,0,0,0,0
1.479121e+09,1201,8,0,0,0,0,0,0,0,0,0
1.479121e+09,497,8,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1.479121e+09,880,8,0,32,0,0,0,0,0,0,0
1.479121e+09,1088,8,255,0,0,0,255,106,8,0,0
1.479121e+09,704,8,20,0,0,0,0,0,0,0,0


In [ ]:
len(normaldf)

988871

In [ ]:
normaldf.to_csv('/content/drive/MyDrive/capstondesign/Car-Hacking Dataset/normal.csv')

In [5]:
rpmdf = pd.read_csv('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/RPM_dataset.csv')

rpmdict = {}

# with open('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/RPM_dataset.csv','r') as file:
#   for line in file.readlines(1080):
#     print(line)

with open('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/RPM_dataset.csv','r') as file:
  for line in file.readlines():
    line = line.split(',')
    if len(line) < 12:
      label = line[-1]
      line = line[:len(line)-1]
      line.extend([0]*(12 - len(line)))
      line[-1] = label
    line[-1] = line[-1].strip()
    rpmdict[line[0]] = line[1:]
# rpmdf = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in rpmdict.items()]))
# rpmdf = rpmdf.T
# rpmdf.head(30)

##RAM 초과 코드
    
# with open('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/RPM_dataset.csv','r') as file:
#   rpmdf = pd.DataFrame(dict([(line[0],pd.Series(line[1:])) for line in file.readlines()]))


## 제대로 안나오는 코드

# new_rpmdf = pd.DataFrame()
# with open('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/RPM_dataset.csv','r') as file:
#   for line in file.readlines():
#     pd.concat([new_rpmdf,pd.DataFrame(dict(zip([line[0]],[pd.Series(line[1:])])))]) 

# new_rpmdf.head(30)

In [6]:
rpmdf = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in rpmdict.items()]))
rpmdf = rpmdf.T
rpmdf.head(100)

,0,1,2,3,4,5,6,7,8,9,10
1478191030.045114,0316,8,05,22,68,09,22,20,00,75,R
1478191030.045353,018f,8,fe,3b,00,00,00,3c,00,00,R
1478191030.045580,0260,8,19,22,22,30,ff,8f,6e,3f,R
1478191030.045815,02a0,8,60,00,83,1d,96,02,bd,00,R
1478191030.046043,0329,8,dc,b8,7e,14,11,20,00,14,R
...,...,...,...,...,...,...,...,...,...,...,...
1478191030.095770,02a0,8,00,00,83,1d,96,02,bd,00,R
1478191030.096004,0329,8,0c,b8,7e,14,11,20,00,14,R
1478191030.096247,04f0,8,00,00,00,80,00,67,d1,13,R
1478191030.096489,0545,8,d8,00,00,84,00,00,00,00,R


In [7]:
columns = ['CAN ID','DLC','Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8','Label']
rpmdf = makeColumns(rpmdf,columns)
rpmdf = rpmdf.fillna(0)
rpmdf = changeDtypes(rpmdf,columns[2:-1])

rpmdf.head(30)

['Data1', 'Data2', 'Data3', 'Data4', 'Data5', 'Data6', 'Data7', 'Data8']


,CAN ID,DLC,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Data8,Label
Timestamp,,,,,,,,,,,
1478191030.045114,790,8,5,34,104,9,34,32,0,117,R
1478191030.045353,399,8,254,59,0,0,0,60,0,0,R
1478191030.045580,608,8,25,34,34,48,255,143,110,63,R
1478191030.045815,672,8,96,0,131,29,150,2,189,0,R
1478191030.046043,809,8,220,184,126,20,17,32,0,20,R
1478191030.046288,1349,8,216,0,0,131,0,0,0,0,R
1478191030.048193,704,8,21,0,0,0,0,0,0,0,R
1478191030.050885,2,8,0,0,0,0,0,7,9,6,R
1478191030.051119,339,8,0,33,16,255,0,255,0,0,R


In [8]:
# is_attack = rpmdf['Label'] == "T"
# rpmdf = rpmdf[is_attack]
len(rpmdf)

4621702

In [9]:
rpmdf.to_csv('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/rpm_total.csv')

In [10]:
geardf = pd.read_csv('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/gear_dataset.csv')
geardf.head(10)

,1478193190.056566,0140,8,00,00.1,00.2,00.3,10,29,2a,24,R
0,1.478193e+09,02c0,8,15,00,00,00,00,00,00,00,R
1,1.478193e+09,0350,8,05,20,44,68,77,00,00,7e,R
2,1.478193e+09,0370,8,00,20,00,00,00,00,00,00,R
3,1.478193e+09,043f,8,10,40,60,ff,78,c4,08,00,R
4,1.478193e+09,0440,8,ff,00,00,00,ff,c4,08,00,R
5,1.478193e+09,0316,8,05,21,68,09,21,21,00,70,R
6,1.478193e+09,018f,8,fe,50,00,00,00,3c,00,00,R
7,1.478193e+09,0002,8,00,00,00,00,00,06,0b,70,R
8,1.478193e+09,0153,8,00,21,10,ff,00,ff,00,00,R
9,1.478193e+09,0260,8,19,21,22,30,08,8e,6d,18,R


In [11]:
geardict ={}
with open('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/gear_dataset.csv','r') as file:
  for line in file.readlines():
    line = line.split(',')
    if len(line) < 12:
      label = line[-1]
      line = line[:len(line)-1]
      line.extend([0]*(12 - len(line)))
      line[-1] = label
    line[-1] = line[-1].strip()
    geardict[line[0]] = line[1:]

In [12]:
geardf = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in geardict.items()]))
geardf = geardf.T
geardf.head(100)

,0,1,2,3,4,5,6,7,8,9,10
1478193190.056566,0140,8,00,00,00,00,10,29,2a,24,R
1478193190.056817,02c0,8,15,00,00,00,00,00,00,00,R
1478193190.057058,0350,8,05,20,44,68,77,00,00,7e,R
1478193190.057304,0370,8,00,20,00,00,00,00,00,00,R
1478193190.057542,043f,8,10,40,60,ff,78,c4,08,00,R
...,...,...,...,...,...,...,...,...,...,...,...
1478193190.111184,04f0,8,00,00,00,80,00,67,d1,13,R
1478193190.113161,0316,8,05,21,70,09,21,21,00,70,R
1478193190.113406,018f,8,fe,50,00,00,00,3c,00,00,R
1478193190.113633,0260,8,19,21,21,30,08,8e,6c,2b,R


In [13]:
columns = ['CAN ID','DLC','Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8','Label']
geardf = makeColumns(geardf,columns)
geardf = geardf.fillna(0)
geardf = changeDtypes(geardf,columns[2:-1])


# is_attack = geardf['Label'] == 'T'
# is_attack = geardf[is_attack]
# is_attack
geardf.head(30)

['Data1', 'Data2', 'Data3', 'Data4', 'Data5', 'Data6', 'Data7', 'Data8']


,CAN ID,DLC,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Data8,Label
Timestamp,,,,,,,,,,,
1478193190.056566,320,8,0,0,0,0,16,41,42,36,R
1478193190.056817,704,8,21,0,0,0,0,0,0,0,R
1478193190.057058,848,8,5,32,68,104,119,0,0,126,R
1478193190.057304,880,8,0,32,0,0,0,0,0,0,R
1478193190.057542,1087,8,16,64,96,255,120,196,8,0,R
1478193190.057786,1088,8,255,0,0,0,255,196,8,0,R
1478193190.063099,790,8,5,33,104,9,33,33,0,112,R
1478193190.063328,399,8,254,80,0,0,0,60,0,0,R
1478193190.063576,2,8,0,0,0,0,0,6,11,112,R


In [14]:
# is_attack = geardf['Label'] == "T"
# geardf = geardf[is_attack]
len(geardf)

4443142

In [15]:
geardf.head(100)

,CAN ID,DLC,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Data8,Label
Timestamp,,,,,,,,,,,
1478193190.056566,320,8,0,0,0,0,16,41,42,36,R
1478193190.056817,704,8,21,0,0,0,0,0,0,0,R
1478193190.057058,848,8,5,32,68,104,119,0,0,126,R
1478193190.057304,880,8,0,32,0,0,0,0,0,0,R
1478193190.057542,1087,8,16,64,96,255,120,196,8,0,R
...,...,...,...,...,...,...,...,...,...,...,...
1478193190.111184,1264,8,0,0,0,128,0,103,209,19,R
1478193190.113161,790,8,5,33,112,9,33,33,0,112,R
1478193190.113406,399,8,254,80,0,0,0,60,0,0,R


In [16]:
geardf.to_csv('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/gear_total.csv')

In [17]:
fuzzydf = pd.read_csv('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/Fuzzy_dataset.csv')
fuzzydict ={}
with open('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/Fuzzy_dataset.csv','r') as file:
  for line in file.readlines():
    line = line.split(',')
    if len(line) < 12:
      label = line[-1]
      line = line[:len(line)-1]
      line.extend([0]*(12 - len(line)))
      line[-1] = label
    line[-1] = line[-1].strip()
    fuzzydict[line[0]] = line[1:]



In [18]:
fuzzydf = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in fuzzydict.items()]))
fuzzydf = fuzzydf.T
fuzzydf.head(100)

,0,1,2,3,4,5,6,7,8,9,10
1478195721.903877,0545,8,d8,00,00,8a,00,00,00,00,R
1478195721.905736,02b0,5,ff,7f,00,05,49,0,0,0,R
1478195721.908437,0002,8,00,00,00,00,00,01,07,15,R
1478195721.908676,0153,8,00,21,10,ff,00,ff,00,00,R
1478195721.909414,0130,8,19,80,00,ff,fe,7f,07,60,R
...,...,...,...,...,...,...,...,...,...,...,...
1478195721.953123,0260,8,18,21,21,30,08,8f,70,06,R
1478195721.953357,02a0,8,04,00,99,1d,97,02,bd,00,R
1478195721.953595,0329,8,40,b8,7f,14,11,20,00,14,R
1478195721.953831,0545,8,d8,55,00,8b,00,00,00,00,R


In [19]:
columns = ['CAN ID','DLC','Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8','Label']
fuzzydf = makeColumns(fuzzydf,columns)
fuzzydf = fuzzydf.fillna(0)
fuzzydf = changeDtypes(fuzzydf,columns[2:-1])

fuzzydf.head(30)

['Data1', 'Data2', 'Data3', 'Data4', 'Data5', 'Data6', 'Data7', 'Data8']


,CAN ID,DLC,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Data8,Label
Timestamp,,,,,,,,,,,
1478195721.903877,1349,8,216,0,0,138,0,0,0,0,R
1478195721.905736,688,5,255,127,0,5,73,0,0,0,R
1478195721.908437,2,8,0,0,0,0,0,1,7,21,R
1478195721.908676,339,8,0,33,16,255,0,255,0,0,R
1478195721.909414,304,8,25,128,0,255,254,127,7,96,R
1478195721.909660,305,8,23,128,0,0,101,127,7,159,R
1478195721.909905,320,8,0,0,0,0,2,32,39,168,R
1478195721.910145,848,8,5,32,20,104,120,0,0,33,R
1478195721.910389,704,8,21,0,0,0,0,0,0,0,R


In [20]:
# is_attack = fuzzydf['Label'] == "T"
# fuzzydf = fuzzydf[is_attack]
len(fuzzydf)

3838860

In [21]:
fuzzydf.to_csv('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/fuzzy_total.csv')

In [5]:
DoSdf = pd.read_csv('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/DoS_dataset.csv')
DoSdict ={}
with open('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/DoS_dataset.csv','r') as file:
  for line in file.readlines():
    line = line.split(',')
    if len(line) < 12:
      line = line[:len(line)-1]
      line.extend([0]*(12 - len(line)))
      line[-1] = 'R'
    line[-1] = line[-1].strip()
    DoSdict[line[0]] = line[1:]


In [6]:
DoSdf = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in DoSdict.items()]))
DoSdf = DoSdf.T
DoSdf.head(100)



,0,1,2,3,4,5,6,7,8,9,10
1478198376.389427,0316,8,05,21,68,09,21,21,00,6f,R
1478198376.389636,018f,8,fe,5b,00,00,00,3c,00,00,R
1478198376.389864,0260,8,19,21,22,30,08,8e,6d,3a,R
1478198376.390096,02a0,8,64,00,9a,1d,97,02,bd,00,R
1478198376.390333,0329,8,40,bb,7f,14,11,20,00,14,R
...,...,...,...,...,...,...,...,...,...,...,...
1478198376.440400,0329,8,86,bb,7f,14,11,20,00,14,R
1478198376.440642,04f0,8,00,00,00,80,00,69,d1,13,R
1478198376.440886,0545,8,d8,00,00,8a,00,00,00,00,R
1478198376.441182,0690,8,00,00,00,00,a0,22,00,00,R


In [7]:
columns = ['CAN ID','DLC','Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8','Label']
DoSdf = makeColumns(DoSdf,columns)
DoSdf = DoSdf.fillna(0)
DoSdf = changeDtypes(DoSdf,columns[2:-1])

# is_attack = DoSdf['Label'] == 'T'
# DoSdf[is_attack].head(100)
DoSdf.head(30)

['Data1', 'Data2', 'Data3', 'Data4', 'Data5', 'Data6', 'Data7', 'Data8']


,CAN ID,DLC,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Data8,Label
Timestamp,,,,,,,,,,,
1478198376.389427,790,8,5,33,104,9,33,33,0,111,R
1478198376.389636,399,8,254,91,0,0,0,60,0,0,R
1478198376.389864,608,8,25,33,34,48,8,142,109,58,R
1478198376.390096,672,8,100,0,154,29,151,2,189,0,R
1478198376.390333,809,8,64,187,127,20,17,32,0,20,R
1478198376.390574,1349,8,216,0,0,138,0,0,0,0,R
1478198376.393317,2,8,0,0,0,0,0,3,11,17,R
1478198376.393552,339,8,0,33,16,255,0,255,0,0,R
1478198376.394175,704,8,20,0,0,0,0,0,0,0,R


In [8]:
# is_dos_attack = DoSdf['Label'] == 'T'
# DoSdf = DoSdf[is_dos_attack]
# # DoSdf.head(10)
len(DoSdf)

3665771

In [9]:
DoSdf.to_csv('/content/drive/My Drive/capstondesign/Car-Hacking Dataset/dos_total.csv')